In [1]:
from thesis_initialise import *

import numpy as _np

In [3]:
from dataclasses import dataclass, fields

In [237]:
################################################################################

from functools import lru_cache as _lru_cache, reduce as _reduce
import numpy as _np

@dataclass
class ChesterWorld:
    nTimes: int = 24
    dtype: str = 'uint8'
    levels: int = 2
    popDensity: int = 1500 # persons per km 2
    scale: float = 2**12 / 1e3 # outermost edge km
    def __post_init__(self):
        self.nAgents = nAgents = round(self.popDensity * self.scale ** 2)
        shape = (nAgents, self.nTimes, self.levels)
        self.agentIDs = _np.arange(nAgents)
        self.schedule = _np.random.randint(0, 2**8, shape, self.dtype)
        self._hashVal = np.random.randint(10**12, 10**13 - 1)
    @_lru_cache
    def get_potential_contacts(self, agentID: int, level: int, /):
        if level:
            shortlist = self.get_potential_contacts(agentID, level - 1)
        else:
            shortlist = self.agentIDs
        schedule = self.schedule[:, :, level][shortlist]
        matches = schedule == schedule[agentID]
        return shortlist[_np.unique(_np.nonzero(matches)[0])]
    def __hash__(self):
        return self._hashVal

################################################################################

#         if level = 0:
#             schedule = self.schedule
#         else:
#             schedule = self.get_potential_contacts(agentID, level - 1)
#         schedule = self.schedule[:, :, :level+1]
#         contacts = _np.nonzero(schedule == schedule[agentID])[0]
#         return sorted(set(contacts))
#     def get_contacts(self, agentID: int):
#         schedule = self.schedule
#         f
#         return _reduce(
#             self.get_potential_contacts,
#             (agentID, )
#             )

In [250]:
%time model = ChesterWorld(levels = 3, scale = 2**16/1e3)
%time contacts = model.get_potential_contacts(0, 0)
%time contacts = model.get_potential_contacts(0, 1)
%time contacts = model.get_potential_contacts(0, 2)
print(f"nContacts = {len(contacts)}")
assert all(_np.any(model.schedule[0] == model.schedule[contact]) for contact in contacts)

CPU times: user 689 ms, sys: 236 ms, total: 925 ms
Wall time: 931 ms
CPU times: user 725 ms, sys: 196 ms, total: 921 ms
Wall time: 921 ms
CPU times: user 95.3 ms, sys: 73 µs, total: 95.4 ms
Wall time: 95.4 ms
CPU times: user 11.6 ms, sys: 0 ns, total: 11.6 ms
Wall time: 11.7 ms
nContacts = 4639
